In [1]:
import jsonlines
import pandas as pd
import numpy as np
import cufflinks as cf
import subprocess
import json
import os
cf.go_offline()
#load_dir = '/Volumes/Timemachine/gcloud/T2/research/analyze-faults/data/json/'
load_dir = '../results'

In [2]:
!../bin/pip install xml2json

You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import json

In [4]:
def process_xml(xml_file):

    # p_name=os.path.splitext(project)[0]
    # table_file=os.path.join(load_dir,p_name+'.csv')
    # res_file=os.path.join(load_dir,p_name+'.estimate.json')
    project_file= "%s.json" % xml_file
    !xml2json -d badgerfish {load_dir}/{xml_file} > {project_file}
    
    killed_mutants=[]
    survived=[]
    f0 = 0
    f1 = 0
    f2 = 0
    with open(project_file) as fp:
        reader = json.load(fp)['mutations']['mutation']
        
    for idx,obj in enumerate(reader):
        if obj['@status']=='KILLED':
            kt = obj['killingTests']['$'].split('|')
            for t in kt:
                killed_mutants.append((idx,t))
            if len(kt) == 1:
                f1 +=1
            elif len(kt) == 2:
                f2 += 1
                
        if obj['@status']=='SURVIVED':
            survived.append(idx)
            f0 +=1

    col_names=['mutant','test']
    df=pd.DataFrame(killed_mutants,columns=col_names)
    m_table=pd.crosstab(df['mutant'],df['test'])
    z = pd.DataFrame(0, index=np.arange(len(survived)), columns=m_table.columns)
    new = m_table.append(z)
    return len(m_table), len(survived),f0, f1, f2, new

In [5]:
projects = [
# 'commons-csv.mutations.xml'
#,'commons-bcel.xml'
#,'commons-cli.xml'
#,'commons-codec.xml'
'commons-collections.mutations.xml'
#,'commons-configuration.xml'
#,'commons-dbcp.xml'
,'commons-dbutils.mutations.xml'
#,'commons-discovery.xml'
,'commons-exec.mutations.xml'
#,'commons-fileupload.xml'
,'commons-imaging.mutations.xml'
,'commons-io.mutations.xml'
# ,'commons-lang.mutations.xml'
#,'commons-mail.xml'
#,'commons-math.mutations.xml'
#,'commons-net.xml'
#,'commons-ognl.xml'
,'commons-pool.mutations.xml'
#,'commons-scxml.xml'
#,'commons-text.xml'
,'commons-validator.mutations.xml'
]

In [6]:
killed, survived, f0, f1, f2, m_table= {}, {}, {}, {}, {}, {}
for project in projects:
    killed[project], survived[project], f0[project], f1[project], f2[project], m_table[project] =process_xml(project)
    print(project)
    print(f"\tNumber of killed mutants: {killed[project]}, survived mutants: {survived[project]}")
    print(f"\tNumber of tests: {m_table[project].shape[1]}")
    print(f"\tF0: {f0[project]}, F1: {f1[project]}, F2: {f2[project]}")
    print()

commons-collections.mutations.xml
	Number of killed mutants: 3462, survived mutants: 357
	Number of tests: 10887
	F0: 357, F1: 1165, F2: 483

commons-dbutils.mutations.xml
	Number of killed mutants: 380, survived mutants: 79
	Number of tests: 291
	F0: 79, F1: 84, F2: 60

commons-exec.mutations.xml
	Number of killed mutants: 212, survived mutants: 71
	Number of tests: 66
	F0: 71, F1: 25, F2: 33

commons-imaging.mutations.xml
	Number of killed mutants: 6156, survived mutants: 2348
	Number of tests: 547
	F0: 2348, F1: 1710, F2: 598

commons-io.mutations.xml
	Number of killed mutants: 2548, survived mutants: 374
	Number of tests: 1253
	F0: 374, F1: 687, F2: 465

commons-pool.mutations.xml
	Number of killed mutants: 705, survived mutants: 281
	Number of tests: 273
	F0: 281, F1: 260, F2: 95

commons-validator.mutations.xml
	Number of killed mutants: 1508, survived mutants: 169
	Number of tests: 489
	F0: 169, F1: 409, F2: 238



In [7]:
for p in projects:
    if p in ['commons-csv.mutations', 'commons-lang.mutations', 'commons-math.mutations']: continue
    p_name=os.path.splitext(p)[0]
    table_file = os.path.join(load_dir,p_name+'.csv')
    res_file=os.path.join(load_dir,p_name+'.estimate.json')
    m_table[p].to_csv(table_file, header=False)
    rscript="predict_population_size.R"
    args=['Rscript',
      rscript,
     '-a',
     '-m',
     table_file,
     '-o',
     res_file]
    print(args)
    sr=subprocess.run(args, capture_output=True)

['Rscript', 'predict_population_size.R', '-a', '-m', '../results/commons-collections.mutations.csv', '-o', '../results/commons-collections.mutations.estimate.json']
['Rscript', 'predict_population_size.R', '-a', '-m', '../results/commons-dbutils.mutations.csv', '-o', '../results/commons-dbutils.mutations.estimate.json']
['Rscript', 'predict_population_size.R', '-a', '-m', '../results/commons-exec.mutations.csv', '-o', '../results/commons-exec.mutations.estimate.json']
['Rscript', 'predict_population_size.R', '-a', '-m', '../results/commons-imaging.mutations.csv', '-o', '../results/commons-imaging.mutations.estimate.json']
['Rscript', 'predict_population_size.R', '-a', '-m', '../results/commons-io.mutations.csv', '-o', '../results/commons-io.mutations.estimate.json']
['Rscript', 'predict_population_size.R', '-a', '-m', '../results/commons-pool.mutations.csv', '-o', '../results/commons-pool.mutations.estimate.json']
['Rscript', 'predict_population_size.R', '-a', '-m', '../results/commons

In [8]:
for p in projects:
    p_name=os.path.splitext(p)[0]
    table_file = os.path.join(load_dir,p_name+'.csv')
    res_file=os.path.join(load_dir,p_name+'.estimate.json')
    print(res_file)
    #m_table = pd.read_csv(table_file, header=None)
    rs = m_table[p].sum(axis=1)
    with open(res_file) as fd:
        res=json.load(fd)
        print("N Mutants:", res['n.mutants'])
        print("N Killed:", res['n.killed'])
        print("N Tests:", res['n.samples'])
        print(res['estimators'][0])
        print(res['estimators'][1])
        print(res['estimators'][2])
        print(res['estimators'][3])
        print(res['estimators'][4])
        print(res['estimators'][5])
        print(res['estimators'][6])
        for i in range(7):
            if not 'lowci' in res['estimators'][i]:
                print(res['estimators'][i]['estimator'], 'doesnt work')
            elif res['estimators'][i]['lowci'] < res['n.mutants']:
                print(res['estimators'][i]['estimator'], 'works')
            else:
                print(res['estimators'][i]['estimator'], 'doesnt work')
    rs.iplot(kind='histogram', bins=m_table[p].shape[1])
    print()

../results/commons-collections.mutations.estimate.json
N Mutants: 3819
N Killed: 3462
N Tests: 10887
{'estimator': 'chao_lower', 'value': 4866.9948, 'lowci': 3658.7054, 'upci': 13497.3663}
{'estimator': 'bootstrap', 'value': 4278.9127, 'lowci': 4195.2727, 'upci': 4362.5528}
{'estimator': 'sample_coverage', 'value': 27653.7046}
{'estimator': 'chao_upperbound', 'value': 4866.9948, 'lowci': 3785.4717, 'upci': 3818.7501}
{'estimator': 'chao2_our', 'value': 4866.9948, 'lowci': 4649.8883, 'upci': 5084.1014}
{'estimator': 'jack_wiqid', 'value': 5899.9778, 'lowci': 5405.418, 'upci': 6520.393}
{'estimator': 'jack_our', 'value': 6008.7602, 'lowci': 5604.4125, 'upci': 6413.1079}
chao_lower works
bootstrap doesnt work
sample_coverage doesnt work
chao_upperbound works
chao2_our doesnt work
jack_wiqid doesnt work
jack_our doesnt work



../results/commons-dbutils.mutations.estimate.json
N Mutants: 459
N Killed: 380
N Tests: 291
{'estimator': 'chao_lower', 'value': 438.8, 'lowci': 387.6602, 'upci': 831.3541}
{'estimator': 'bootstrap', 'value': 452.1133, 'lowci': 431.3333, 'upci': 472.8933}
{'estimator': 'sample_coverage', 'value': 634.7838, 'lowci': 503.0391, 'upci': 766.5284}
{'estimator': 'chao_upperbound', 'value': 438.8, 'lowci': 397.7054, 'upci': 456.8088}
{'estimator': 'chao2_our', 'value': 438.8, 'lowci': 405.9407, 'upci': 471.6593}
{'estimator': 'jack_wiqid', 'value': 465.1814, 'lowci': 416.6217, 'upci': 578.1304}
{'estimator': 'jack_our', 'value': 487.7519, 'lowci': 443.9658, 'upci': 531.5379}
chao_lower works
bootstrap works
sample_coverage doesnt work
chao_upperbound works
chao2_our works
jack_wiqid works
jack_our works



../results/commons-exec.mutations.estimate.json
N Mutants: 283
N Killed: 210
N Tests: 66
{'estimator': 'chao_lower', 'value': 219.4697, 'lowci': 211.017, 'upci': 298.1744}
{'estimator': 'bootstrap', 'value': 405486757861.751, 'lowci': -8736773498369.76, 'upci': 9547747014093.26}
{'estimator': 'sample_coverage', 'value': 231.7111, 'lowci': 220.9494, 'upci': 242.4728}
{'estimator': 'chao_upperbound', 'value': 219.4697, 'lowci': 211.7768, 'upci': 255.3315}
{'estimator': 'chao2_our', 'value': 219.4697, 'lowci': 209.3733, 'upci': 229.5661}
{'estimator': 'jack_wiqid', 'value': 232.4546, 'lowci': 217.86, 'upci': 274.1485}
{'estimator': 'jack_our', 'value': 234.6212, 'lowci': 220.9195, 'upci': 248.3229}
chao_lower works
bootstrap works
sample_coverage works
chao_upperbound works
chao2_our works
jack_wiqid works
jack_our works



../results/commons-imaging.mutations.estimate.json
N Mutants: 8504
N Killed: 6156
N Tests: 547
{'estimator': 'chao_lower', 'value': 8600.8997, 'lowci': 6498.8977, 'upci': 23588.4138}
{'estimator': 'bootstrap', 'value': 7359.0541, 'lowci': 6899.9546, 'upci': 7818.1537}
{'estimator': 'sample_coverage', 'value': 17006.4526}
{'estimator': 'chao_upperbound', 'value': 8600.8997, 'lowci': 7789.264, 'upci': 8493.2576}
{'estimator': 'chao2_our', 'value': 8600.8997, 'lowci': 8282.2968, 'upci': 8919.5026}
{'estimator': 'jack_wiqid', 'value': 11495.5677, 'lowci': 10749.347, 'upci': 12363.017}
{'estimator': 'jack_our', 'value': 11629.6897, 'lowci': 10843.4474, 'upci': 12415.932}
chao_lower works
bootstrap works
sample_coverage doesnt work
chao_upperbound works
chao2_our works
jack_wiqid doesnt work
jack_our doesnt work



../results/commons-io.mutations.estimate.json
N Mutants: 2922
N Killed: 2548
N Tests: 1253
{'estimator': 'chao_lower', 'value': 3055.4935, 'lowci': 2618.7932, 'upci': 6186.0549}
{'estimator': 'bootstrap', 'value': 3106.4183, 'lowci': 3032.266, 'upci': 3180.5707}
{'estimator': 'sample_coverage', 'value': 6357.8927}
{'estimator': 'chao_upperbound', 'value': 3055.4935, 'lowci': 2819.8345, 'upci': 2920.9019}
{'estimator': 'chao2_our', 'value': 3055.4935, 'lowci': 2956.3066, 'upci': 3154.6805}
{'estimator': 'jack_wiqid', 'value': 3437.8805, 'lowci': 3262.8779, 'upci': 3655.7237}
{'estimator': 'jack_our', 'value': 3456.4682, 'lowci': 3330.7732, 'upci': 3582.1632}
chao_lower works
bootstrap doesnt work
sample_coverage doesnt work
chao_upperbound works
chao2_our doesnt work
jack_wiqid doesnt work
jack_our doesnt work



../results/commons-pool.mutations.estimate.json
N Mutants: 986
N Killed: 696
N Tests: 273
{'estimator': 'chao_lower', 'value': 1051.7895, 'lowci': 744.7347, 'upci': 3293.4542}
{'estimator': 'bootstrap', 'value': 885.4787, 'lowci': 849.7274, 'upci': 921.23}
{'estimator': 'sample_coverage', 'value': 2726.1192, 'lowci': 2241.4674, 'upci': 3210.7709}
{'estimator': 'chao_upperbound', 'value': 1051.7895, 'lowci': 824.3298, 'upci': 983.2888}
{'estimator': 'chao2_our', 'value': 1051.7895, 'lowci': 933.6069, 'upci': 1169.972}
{'estimator': 'jack_wiqid', 'value': 1259.9228, 'lowci': 1085.918, 'upci': 1511.5791}
{'estimator': 'jack_our', 'value': 1332.8353, 'lowci': 1147.1861, 'upci': 1518.4846}
chao_lower works
bootstrap works
sample_coverage doesnt work
chao_upperbound works
chao2_our works
jack_wiqid doesnt work
jack_our doesnt work



../results/commons-validator.mutations.estimate.json
N Mutants: 1677
N Killed: 1508
N Tests: 489
{'estimator': 'chao_lower', 'value': 1859.4307, 'lowci': 1556.6957, 'upci': 4044.2307}
{'estimator': 'bootstrap', 'value': 1825.5361, 'lowci': 1746.8148, 'upci': 1904.2575}
{'estimator': 'sample_coverage', 'value': 2909.9631, 'lowci': 2711.1822, 'upci': 3108.7439}
{'estimator': 'chao_upperbound', 'value': 1859.4307, 'lowci': 1630.6229, 'upci': 1676.5623}
{'estimator': 'chao2_our', 'value': 1859.4307, 'lowci': 1770.0794, 'upci': 1948.782}
{'estimator': 'jack_wiqid', 'value': 2110.1708, 'lowci': 1965.517, 'upci': 2300.5599}
{'estimator': 'jack_our', 'value': 2171.9545, 'lowci': 2013.0815, 'upci': 2330.8276}
chao_lower works
bootstrap doesnt work
sample_coverage doesnt work
chao_upperbound works
chao2_our doesnt work
jack_wiqid doesnt work
jack_our doesnt work
